In [ ]:
#import
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
import joblib
from lightgbm import LGBMRegressor
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [ ]:
df = pd.read_csv("/content/fake_job_postings.csv",error_bad_lines=False ,engine="python")
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 3406: unexpected end of data


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3404 entries, 0 to 3403
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               3404 non-null   int64 
 1   title                3404 non-null   object
 2   location             3358 non-null   object
 3   department           1254 non-null   object
 4   salary_range         538 non-null    object
 5   company_profile      2698 non-null   object
 6   description          3404 non-null   object
 7   requirements         2939 non-null   object
 8   benefits             2038 non-null   object
 9   telecommuting        3404 non-null   int64 
 10  has_company_logo     3404 non-null   int64 
 11  has_questions        3404 non-null   int64 
 12  employment_type      2627 non-null   object
 13  required_experience  2044 non-null   object
 14  required_education   1914 non-null   object
 15  industry             2430 non-null   object
 16  functi

In [ ]:
# replace incorrect values and fill missing values
df.salary_range = df.salary_range.fillna("0-0")

In [ ]:
df.salary_range.replace({'40000':'40000-40000','Oct-15':'102-102','9-Dec':'102-102','3-Apr':'102-102','4-Apr':'102-102','8-Sep':'102-102','4-Jun':'102-102','10-Oct':'102-102','Oct-20':'102-102','Jun-18':'102-102','11-Nov':'102-102','11-Dec':'102-102','2-Apr':'102-102','2-Jun':'102-102','Dec-25':'102-102','10-Nov':'102-102'}, inplace=True)

In [ ]:
df.salary_range.replace({'102-102':'0-0'}, inplace=True)

In [ ]:
df.location.fillna("NO LOCATION", inplace=True)
df.department.fillna("NO DEPARTMENT", inplace=True)
df.company_profile.fillna("NO PROFILE", inplace=True)
df.description.fillna("NO DESCRIPTION", inplace=True)
df.requirements.fillna("NO REQUIREMENTS", inplace=True)
df.benefits.fillna("NO BENEFITS", inplace=True)
df.employment_type.fillna("NO EMP_TYPE", inplace=True)
df.required_experience.fillna("NO EXPERIENCE", inplace=True)
df.required_education.fillna("NO EDUCATION", inplace=True)
df.industry.fillna("NO INDUSTRY", inplace=True)
df.function.fillna("NO FUNCTIONS", inplace=True)
df.telecommuting.replace({0:"NO telecommuting",1:"YES telecommuting"},inplace=True)
df.has_company_logo.replace({0:"NO logo",1:'YES logo'},inplace=True)
df.has_questions.replace({0:"NO questions",1:"Yes question"},inplace=True)

In [ ]:
#Function to get probability of output from each columns¶
# used count vectorizer on the strings and then used logistic regression on it to get the predicted probability

# function to get probability of efeatures

def get_feat_prob(X,y):
    # vectorize
    vec = CountVectorizer(max_features=10000)
    vec.fit(X)
    trn,tst,trn_o,tst_o = train_test_split(X, y, test_size=0.4, random_state=20)
    # transform
    trn_abs = vec.transform(trn)
    tst_abs = vec.transform(tst)
    X_abs = vec.transform(X)
    # fit to model
    clf = LogisticRegression(C = 10, n_jobs=-1)
    clf.fit(trn_abs, trn_o)
    # predict score
    tst_preds = clf.predict(tst_abs)
    print("score: ", f1_score(tst_o, tst_preds, average='micro'))
    print("confusion matrix: \n\n", confusion_matrix(tst_o, tst_preds))
    # predict probabilities and make a dataframe of 1's probability
    all_preds = clf.predict_proba(X_abs)
    all_preds_df = pd.DataFrame(all_preds, columns=['zero','one'])
          
    return all_preds_df['one']
#     return clf
y = df['fraudulent']

In [ ]:
#check for each of the categorical columns first
each_proba = []
for col in df.iloc[:,:18]:
    if df[col].dtype == 'object':
        print("\n-----------",col,"-----------")
        each_proba.append(get_feat_prob(df[col],y))


----------- title -----------
score:  0.9647577092511013
confusion matrix: 

 [[1308   11]
 [  37    6]]

----------- location -----------
score:  0.9720998531571219
confusion matrix: 

 [[1318    1]
 [  37    6]]

----------- department -----------
score:  0.9706314243759178
confusion matrix: 

 [[1319    0]
 [  40    3]]

----------- salary_range -----------
score:  0.9684287812041116
confusion matrix: 

 [[1318    1]
 [  42    1]]

----------- company_profile -----------
score:  0.9838472834067548
confusion matrix: 

 [[1319    0]
 [  22   21]]

----------- description -----------
score:  0.9765051395007343
confusion matrix: 

 [[1315    4]
 [  28   15]]

----------- requirements -----------
score:  0.9706314243759178
confusion matrix: 

 [[1311    8]
 [  32   11]]

----------- benefits -----------
score:  0.9765051395007343
confusion matrix: 

 [[1317    2]
 [  30   13]]

----------- telecommuting -----------
score:  0.9684287812041116
confusion matrix: 

 [[1319    0]
 [  43    0

In [ ]:
# make list of df's into single dataframe
each_proba_df = pd.concat(each_proba[:7] ,axis=1)

In [ ]:
each_proba_df.columns = df.columns[1:8]

In [ ]:
each_proba_df.head()

,title,location,department,salary_range,company_profile,description,requirements
0,0.000801,0.000599,0.002170,0.027082,0.000035,7.923887e-05,0.000087
1,0.010698,0.002461,0.008703,0.027082,0.000140,7.652571e-09,0.000005
2,0.019102,0.009991,0.034510,0.027082,0.000005,7.152032e-03,0.000266
3,0.013134,0.022881,0.011791,0.027082,0.000126,2.419607e-08,0.000045
4,0.007955,0.015533,0.034510,0.027082,0.000042,4.539576e-05,0.000025


In [ ]:
left = df.iloc[:,9:-3]
left.head()

X_left = pd.get_dummies(left)
y = df.fraudulent

X_trainl,X_testl,y_trainl,y_testl = train_test_split(X_left,y, test_size=0.3,random_state=42)

clf_left = XGBClassifier()
clf_left.fit(X_trainl,y_trainl)
print(clf_left.score(X_testl,y_testl))
print(confusion_matrix(y_testl,clf_left.predict(X_testl)))

X_col_left = pd.DataFrame(clf_left.predict(X_left))

0.9657534246575342
[[987   0]
 [ 35   0]]


In [ ]:
#The below function is uses clustering approach to cluster our data into how many ever groups it can  and apply the passes models onto each cluster so that we can see what works best

def clustering_approach(X,y, models,type = "none"):    
    dfs = {}
    X_cls = {}
    y_cls = {}
    X_scaled = {}
    X_train, X_test, y_train, y_test = {},{},{},{}
    y_pred = {}
    models_out = {}
    
    # create knn model and predict
    knn_clf = KNeighborsClassifier()
    knn_clf.fit(X,y)
    df = pd.concat([X,y],axis=1) # so we can later separate x and y for each cluster
    df['knn_clf'] = knn_clf.predict(X)
    no_cls = knn_clf.classes_
    
    # get the dataframes, apply std.scaler, form train, test sets, apply models
    for cls in knn_clf.classes_:
        print("--------------The {} cluster's results-------------------".format(cls),end="\n\n")
        dfs[cls] = df[df['knn_clf'] == cls].iloc[:,:-1]
        
        X_cls[cls] = dfs[cls].iloc[:,:-1]
        y_cls[cls] = dfs[cls].iloc[:,-1]
        scaler = StandardScaler()
        X_scaled[cls] = scaler.fit_transform(X_cls[cls])
#         X_scaled[cls] = pd.DataFrame(X_scaled[cls],columns=df.columns[:-1])
        
        X_train[cls],X_test[cls],y_train[cls],y_test[cls] = train_test_split(X_scaled[cls],y_cls[cls],test_size=0.4,random_state=42)
        print(y_train[cls].value_counts())
        print(y_test[cls].value_counts())
        
        # type can be used for analyzing... eg: confusion matrix
        for model in models:
            model.fit(X_train[cls], y_train[cls])
            y_pred[cls] = model.predict(X_test[cls])
            print(model)
            print(model.score(X_test[cls],y_test[cls]))
            print(confusion_matrix(y_pred[cls], y_test[cls]), end="\n\n")
            print((cross_val_score(model,X_scaled[cls],y_cls[cls])).mean())
            models_out[str(model) + str(cls)] = model
        
    return [X_train, X_test, y_train, y_test,knn_clf, models_out]

In [ ]:
X_new = pd.concat([each_proba_df,X_col_left], axis=1)

In [ ]:
models = [LogisticRegression(), RandomForestClassifier(), KNeighborsClassifier(), XGBClassifier(verbosity = 0),LGBMClassifier(),SVC(), GaussianNB()] 

X_train, X_test, y_train, y_test,clusterer, models = clustering_approach(X_new,y,models)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


--------------The 0 cluster's results-------------------

0    1972
1      12
Name: fraudulent, dtype: int64
0    1318
1       5
Name: fraudulent, dtype: int64
LogisticRegression()
0.9962207105064248
[[1318    5]
 [   0    0]]



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.9942552481592022
RandomForestClassifier()
0.9962207105064248
[[1318    5]
 [   0    0]]

0.9948599348236445
KNeighborsClassifier()
0.9962207105064248
[[1318    5]
 [   0    0]]

0.9948599348236445
XGBClassifier(verbosity=0)
0.9962207105064248
[[1318    5]
 [   0    0]]

0.9948599348236445
LGBMClassifier()
0.9962207105064248
[[1318    5]
 [   0    0]]

0.9945578200200191
SVC()
0.9962207105064248
[[1318    5]
 [   0    0]]

0.9948599348236445
GaussianNB()
0.963718820861678
[[1274    4]
 [  44    1]]

0.9609883404710431
--------------The 1 cluster's results-------------------

1    58
Name: fraudulent, dtype: int64
1    38
0     1
Name: fraudulent, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


ValueError: ignored

In [ ]:
m = [[6577 +3, 2+11],[ 218+9, 312+20]]

In [ ]:
(m[0][0]+m[1][1])/(m[0][0]+m[0][1]+m[1][1]+m[1][0])

0.9664429530201343

In [ ]:
(m[1][1])/(m[0][1]+m[1][1])

0.9623188405797102